# Solución de problemas de programación lineal usando R
Notas de clase sobre la planificación centralizada de mercados eléctricos

**Juan David Velásquez Henao**   
jdvelasq@unal.edu.co  
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

[Licencia](https://github.com/jdvelasq/ETVL-R/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/ETVL-R/blob/master/Readme.md)

**Software utilizado**.

> Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org), en el cual se presenta un tutorial sobre la extracción, transformación, visualización y carga de datos usando **R** en el contexto de la ciencia de los datos. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Linux y OS X.

> Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

> Haga clic [aquí](http://nbviewer.jupyter.org/github/jdvelasq/metodos-y-modelos/blob/master/01-06-actividad-solucion-PL-usando-R.ipynb) para ver la última versión de este documento en nbviewer.

> Descargue la última versión de este documento a su disco duro; luego, carguelo y ejecutelo en línea en [Try Jupyter!](https://try.jupyter.org)


#### Contenido

> * [Librería utilizada](#Librería-utilizada)
* [Ejemplos](#Ejemplos)
   * [Ejemplo 1](#Ejemplo-1)
   * [Ejemplo 2](#Ejemplo-2)

# Librería utilizada

[Contenido](#Contenido)

Para realizar la solución de los modelos de programación lineal se usará la libreraía lp-solve.

In [1]:
# solo se requiere una vez para instalar las librerias
# install.packages("lpSolve")
# install.packages("lpSolveAPI")


The downloaded binary packages are in
	/var/folders/4h/h8f6php55ng8t72qj45bhkhr0000gn/T//Rtmp3rFyNh/downloaded_packages

The downloaded binary packages are in
	/var/folders/4h/h8f6php55ng8t72qj45bhkhr0000gn/T//Rtmp3rFyNh/downloaded_packages


# Ejemplos

[Contenido](#Contenido)

## Ejemplo 1

[Contenido](#Contenido)

In [2]:
library(lpSolveAPI)  # solo se requiere una vez en cada sesión

$$\text{max} ~ 0.3 x_1 + x_2$$

Sujeto a:

$$x_1 + x_2 <= 2$$
$$-x_1 + x_2 <= 1$$
$$x_1, x_2 >= 0$$

In [3]:
m <- make.lp(nrow = 0, ncol = 2)        # número de filas y columnas de las restricciones
                                        # se crean 0 restricciones (nrow = 0), y se adicionan
                                        # via la función add.constraint()

# función objetivo: min -0.3 x1 - x2
set.objfn(m, c(-0.3, -1))         

# restricción 1:  x1 + x2 <= 2
add.constraint(m, c( 1, 1), "<=", 2)    

# restricción 2: −x1 + x2 <= 1
add.constraint(m, c(-1, 1), "<=", 1)    

RowNames <- c("R1", "R2")               # nombres de restricciones
ColNames <- c("x1", "x2")               # nombres de variables
dimnames(m) <- list(RowNames, ColNames) # asignación de nombres al modelo

Visualización del modelo:

In [4]:
m

Model name: 
            x1    x2       
Minimize  -0.3    -1       
R1           1     1  <=  2
R2          -1     1  <=  1
Kind       Std   Std       
Type      Real  Real       
Upper      Inf   Inf       
Lower        0     0       

In [5]:
# Llama el solver.
# La función devuelve un código de status:
#   0:   "optimal solution found"
#   1:   "the model is sub-optimal"
#   2:   "the model is infeasible"
#   3:   "the model is unbounded"
#   4:   "the model is degenerate"
#   5:   "numerical failure encountered"
#   6:   "process aborted"
#   7:   "timeout"
#   9:   "the model was solved by presolve"
#   10:  "the branch and bound routine failed"
#   11:  "the branch and bound was stopped because of a break-at-first or break-at-value"
#   12:  "a feasible branch and bound solution was found"
#   13:  "no feasible branch and bound solution was found"
solve(m)

[1] 0

In [6]:
get.objective(m)     # valor de la función objetivo

[1] -1.65

In [7]:
get.variables(m)    # valores óptimos de las variables

[1] 0.5 1.5

In [8]:
get.constraints(m)  # valores de las restricciones

[1] 2 1

## Ejemplo 2

[Contenido](#Contenido)

Este problema de optimización incorpora cotas en las variables (dos últimas restricciones).

Función objetivo:  
$$\text{minimize } x_1 + 3~x_2 + 6.24~x_3 + 0.1~x_4$$  
Sujeto a:  
$$78.26~x_2 + 2.9~x_4 >= 92.3$$  
$$ 0.24~x_1 + 11.31~x_3 <= 14.8$$  
$$ 12.68~x_1 + 0.08~x_3 + 0.9~
x_4 >= 4$$  
$$ 18 <= x_4 <= 48.98$$  
$$ x_1 >= 28.6$$  

In [9]:
# modelo
m <- make.lp(0, 4)

# función objetivo: min x1 + 3 x2 + 6.24 x3 + 0.1 x4
set.objfn(m, c(1, 3, 6.24, 0.1))

# restricción 1: 78.26 x2 + 2.9 x4 >= 92.3
add.constraint(m, c(0, 78.26, 0, 2.9), ">=", 92.3)

# restricción 2: 0.24 x1 + 11.31 x3 <= 14.8
add.constraint(m, c(0.24, 0, 11.31, 0), "<=", 14.8)

# restricción 3: 12.68 x1 + 0.08 x3 + 0.9 x4 >= 4
add.constraint(m, c(12.68, 0, 0.08, 0.9), ">=", 4)

# límites inferiores para las variables (diferentes de cero)
# 28.6 <= x1
# 18 <= x4
set.bounds(m, lower = c(28.6, 18), columns = c(1, 4))

# límites superiores para las variables (diferentes de infinito)
set.bounds(m, upper = 48.98, columns = 4)

# nombres de las filas y las columnas
RowNames <- c("R1", "R2", "R3")
ColNames <- c("x1", "x2", "x3", "x4")
dimnames(m) <- list(RowNames, ColNames)

In [10]:
# visualización del modelo
m

Model name: 
             x1     x2     x3     x4          
Minimize      1      3   6.24    0.1          
R1            0  78.26      0    2.9  >=  92.3
R2         0.24      0  11.31      0  <=  14.8
R3        12.68      0   0.08    0.9  >=     4
Kind        Std    Std    Std    Std          
Type       Real   Real   Real   Real          
Upper       Inf    Inf    Inf  48.98          
Lower      28.6      0      0     18          

In [11]:
# solución
solve(m)   # 0:- "optimal solution found"

[1] 0

In [12]:
# valor de la función objetivo en el óptimo
get.objective(m)

[1] 31.78276

In [13]:
get.variables(m)

[1] 28.60000  0.00000  0.00000 31.82759

In [14]:
# valores de las restricciones para la solución óptima
get.constraints(m)

[1]  92.3000   6.8640 391.2928

**Ejercicio.--** Solucione los siguientes modelos de programación lineal.

**Problema 1**

Función objetivo:  
$$\text{maximize } 2~x_1 + 3~x_2$$  
Sujeto a:  
$$x_1 + 2~x_2 <= 6$$  
$$2~x_1 + x_2 <= 8$$  
$$ x_1 >= 8$$  
$$x_1, x_2 >= 0$$

In [15]:
#
# escriba su código aquí
#

**Problema 2**

Función objetivo:  
$$\text{maximize } 3~x_1 + 5~x_2$$  
Sujeto a:  
$$x_1  >= 5$$  
$$x_2 <= 10$$  
$$ x_1 + 2~x_2 >= 10$$  
$$x_1, x_2 >= 0$$

In [16]:
#
# escriba su código aquí
#

**Problema 3**

Función objetivo:  
$$\text{maximize } 40~x_1 + 60~x_2$$  
Sujeto a:  
$$2x_1 + x_2  <= 70$$  
$$x_1 + x_2 <= 40$$  
$$ x_1 + 3~x_2 <= 90$$  
$$x_1, x_2 >= 0$$

In [17]:
#
# escriba su código aquí
#

**Problema 4**

Función objetivo:  
$$\text{maximize } 3~x_1 + 2~x_2$$  
Sujeto a:  
$$6~x_1 + 4~ x_2  <= 70$$  
$$ x_1 <= 3$$  
$$x_1, x_2 >= 0$$

In [18]:
#
# escriba su código aquí
#

---

[Contenido](#Contenido)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licencia de Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />Este obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">licencia de Creative Commons Reconocimiento-NoComercial-SinObraDerivada 4.0 Internacional</a>.